In [1]:
#import all the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn import preprocessing
%pylab inline

# To ignore some warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Load ACS and garden data
ACS = pd.read_csv("../data/ACS_Merge_Norm_Final.csv",index_col=0)
# Load crime and garden data
crime = pd.read_csv("../data/Crime_totals_normalized.csv",index_col=0)

In [3]:
ACS.head()

,GeoType,NTA,GeoID,Borough,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,...,Pop16yrsOver,Labor_forceN,CivilianLF,CLF_UnemployedN,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden
0,NTA2010,Bath Beach,BK27,Brooklyn,33163.0,0.503151,0.496849,0.214184,0.147393,0.176733,...,26855.0,0.596016,15985.0,0.070566,23822.0,0.663798,0.224372,0.111829,0.0,0.0
1,NTA2010,Bay Ridge,BK31,Brooklyn,79134.0,0.478037,0.521963,0.190942,0.156444,0.178608,...,65479.0,0.644115,42057.0,0.074922,58178.0,0.535735,0.271976,0.192289,0.0,0.0
2,NTA2010,Bedford,BK75,Brooklyn,75318.0,0.484492,0.515508,0.297127,0.077219,0.194933,...,54904.0,0.673776,36960.0,0.125027,44766.0,0.678193,0.204352,0.117455,23.0,1.0
3,NTA2010,Bensonhurst East,BK29,Brooklyn,64267.0,0.486766,0.513234,0.198749,0.157266,0.123283,...,52857.0,0.605483,31983.0,0.085202,46010.0,0.698153,0.204564,0.097283,1.0,1.0
4,NTA2010,Bensonhurst West,BK28,Brooklyn,91646.0,0.487212,0.512788,0.210462,0.142625,0.145975,...,74470.0,0.615577,45833.0,0.088342,65016.0,0.737618,0.188800,0.073582,2.0,1.0


In [4]:
ACS.columns

Index(['GeoType', 'NTA', 'GeoID', 'Borough', 'Pop_1E', 'MaleEN', 'FemEN',
       'PopUnd18N', 'PopOv65N', 'HispN', 'NotHispN', 'WhiteN', 'BlackN',
       'NativeN', 'AsianN', 'PacificN', 'OtherRace1N', 'OtherRace2N', 'HHPop',
       'HHINC_U10N', 'HHINC_10t14N', 'HHINC_15t24N', 'HHINC_25t34N',
       'HHINC_35t49N', 'HHINC_50t74N', 'HHINC_75t99N', 'HHINC_100t149N',
       'HHINC_150t199N', 'HHINC_200plN', 'Pop16yrsOver', 'Labor_forceN',
       'CivilianLF', 'CLF_UnemployedN', 'PopOver25', 'EduOtherN',
       'EduBachelor', 'EduGradProf', 'GardenCounts', 'HasGarden'],
      dtype='object')

In [5]:
ACS = ACS.set_index('NTA')
ACS.head()

,GeoType,GeoID,Borough,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,NotHispN,...,Pop16yrsOver,Labor_forceN,CivilianLF,CLF_UnemployedN,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden
NTA,,,,,,,,,,,,,,,,,,,,,
Bath Beach,NTA2010,BK27,Brooklyn,33163.0,0.503151,0.496849,0.214184,0.147393,0.176733,0.823267,...,26855.0,0.596016,15985.0,0.070566,23822.0,0.663798,0.224372,0.111829,0.0,0.0
Bay Ridge,NTA2010,BK31,Brooklyn,79134.0,0.478037,0.521963,0.190942,0.156444,0.178608,0.821392,...,65479.0,0.644115,42057.0,0.074922,58178.0,0.535735,0.271976,0.192289,0.0,0.0
Bedford,NTA2010,BK75,Brooklyn,75318.0,0.484492,0.515508,0.297127,0.077219,0.194933,0.805067,...,54904.0,0.673776,36960.0,0.125027,44766.0,0.678193,0.204352,0.117455,23.0,1.0
Bensonhurst East,NTA2010,BK29,Brooklyn,64267.0,0.486766,0.513234,0.198749,0.157266,0.123283,0.876717,...,52857.0,0.605483,31983.0,0.085202,46010.0,0.698153,0.204564,0.097283,1.0,1.0
Bensonhurst West,NTA2010,BK28,Brooklyn,91646.0,0.487212,0.512788,0.210462,0.142625,0.145975,0.854025,...,74470.0,0.615577,45833.0,0.088342,65016.0,0.737618,0.188800,0.073582,2.0,1.0


In [6]:
ACS = ACS.drop(columns=['GeoID','GeoType','Borough'])
ACS.head()

,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,NotHispN,WhiteN,BlackN,NativeN,...,Pop16yrsOver,Labor_forceN,CivilianLF,CLF_UnemployedN,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden
NTA,,,,,,,,,,,,,,,,,,,,,
Bath Beach,33163.0,0.503151,0.496849,0.214184,0.147393,0.176733,0.823267,0.436933,0.012182,0.000332,...,26855.0,0.596016,15985.0,0.070566,23822.0,0.663798,0.224372,0.111829,0.0,0.0
Bay Ridge,79134.0,0.478037,0.521963,0.190942,0.156444,0.178608,0.821392,0.634329,0.019094,0.002287,...,65479.0,0.644115,42057.0,0.074922,58178.0,0.535735,0.271976,0.192289,0.0,0.0
Bedford,75318.0,0.484492,0.515508,0.297127,0.077219,0.194933,0.805067,0.350607,0.405826,0.000876,...,54904.0,0.673776,36960.0,0.125027,44766.0,0.678193,0.204352,0.117455,23.0,1.0
Bensonhurst East,64267.0,0.486766,0.513234,0.198749,0.157266,0.123283,0.876717,0.465153,0.008262,0.000000,...,52857.0,0.605483,31983.0,0.085202,46010.0,0.698153,0.204564,0.097283,1.0,1.0
Bensonhurst West,91646.0,0.487212,0.512788,0.210462,0.142625,0.145975,0.854025,0.426773,0.008740,0.001211,...,74470.0,0.615577,45833.0,0.088342,65016.0,0.737618,0.188800,0.073582,2.0,1.0


In [7]:
crime.head()

,FELONY,MISDEMEANOR,VIOLATION,TOTAL_CRIME,Population,GardenCounts,FELONY_norm,MISDEMEANOR_norm,VIOLATION_norm,TOTAL_CRIME_norm
ntaname,,,,,,,,,,
Allerton-Pelham Gardens,752.0,1925.0,444.0,3121.0,32450.0,1.0,0.023174,0.059322,0.013683,0.096179
Annadale-Huguenot-Prince's Bay-Eltingville,431.0,1215.0,464.0,2110.0,28698.0,0.0,0.015018,0.042337,0.016168,0.073524
Arden Heights,263.0,785.0,324.0,1372.0,26482.0,0.0,0.009931,0.029643,0.012235,0.051809
Astoria,4002.0,8082.0,2211.0,14295.0,77252.0,1.0,0.051804,0.104619,0.028621,0.185044
Auburndale,557.0,713.0,258.0,1528.0,20609.0,0.0,0.027027,0.034597,0.012519,0.074142


In [8]:
# merge crime and ACS data
data = ACS.merge(crime[['FELONY_norm', 'MISDEMEANOR_norm', 'VIOLATION_norm', 'TOTAL_CRIME_norm']], how='left', left_index=True, right_index = True)
data.head()

,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,NotHispN,WhiteN,BlackN,NativeN,...,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden,FELONY_norm,MISDEMEANOR_norm,VIOLATION_norm,TOTAL_CRIME_norm
NTA,,,,,,,,,,,,,,,,,,,,,
Bath Beach,33163.0,0.503151,0.496849,0.214184,0.147393,0.176733,0.823267,0.436933,0.012182,0.000332,...,23822.0,0.663798,0.224372,0.111829,0.0,0.0,0.030033,0.065645,0.014444,0.110123
Bay Ridge,79134.0,0.478037,0.521963,0.190942,0.156444,0.178608,0.821392,0.634329,0.019094,0.002287,...,58178.0,0.535735,0.271976,0.192289,0.0,0.0,0.033627,0.082051,0.023833,0.139510
Bedford,75318.0,0.484492,0.515508,0.297127,0.077219,0.194933,0.805067,0.350607,0.405826,0.000876,...,44766.0,0.678193,0.204352,0.117455,23.0,1.0,0.082026,0.124671,0.032993,0.239690
Bensonhurst East,64267.0,0.486766,0.513234,0.198749,0.157266,0.123283,0.876717,0.465153,0.008262,0.000000,...,46010.0,0.698153,0.204564,0.097283,1.0,1.0,0.029377,0.048003,0.014969,0.092349
Bensonhurst West,91646.0,0.487212,0.512788,0.210462,0.142625,0.145975,0.854025,0.426773,0.008740,0.001211,...,65016.0,0.737618,0.188800,0.073582,2.0,1.0,0.027704,0.045981,0.013105,0.086790


In [9]:
# check for nulls
data[data.isnull().any(axis=1)]
# probably want to drop airport and park-cemetery-etc-Staten Island	 since they have population=0
# want to fillna for Hudson Yards-Chelsea-Flat Iron-Union Square
# not sure why [Rikers Island, park-cemetery-etc-Manhattan] have null values

,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,NotHispN,WhiteN,BlackN,NativeN,...,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden,FELONY_norm,MISDEMEANOR_norm,VIOLATION_norm,TOTAL_CRIME_norm
NTA,,,,,,,,,,,,,,,,,,,,,
Hudson Yards-Chelsea-Flat Iron-Union Square,77205.0,0.520174,0.479826,0.103983,0.115472,0.140380,0.859620,0.631449,0.057367,0.000661,...,62879.0,0.255793,0.398273,0.345934,0.0,0.0,NaN,NaN,NaN,NaN
park-cemetery-etc-Manhattan,1765.0,0.877620,0.122380,0.003399,0.094051,0.202833,0.797167,0.218697,0.511615,0.007932,...,1677.0,0.938581,0.042338,0.019082,2.0,1.0,0.760340,1.542210,0.287252,2.589802
Airport,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
park-cemetery-etc-Staten Island,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
Rikers Island,7503.0,0.932160,0.067840,0.035586,0.004398,0.298547,0.701453,0.067973,0.599227,0.001866,...,5322.0,0.985720,0.006576,0.007704,0.0,0.0,0.411036,0.278822,0.002532,0.692390


In [10]:
data.shape

(195, 39)

In [11]:
data.drop(index=['Airport', 'park-cemetery-etc-Staten Island'], inplace=True)
data.shape

(193, 39)

In [12]:
# assign crime nan to zero
data[['FELONY_norm', 'MISDEMEANOR_norm', 'VIOLATION_norm', 'TOTAL_CRIME_norm']] = data[['FELONY_norm', 'MISDEMEANOR_norm', 'VIOLATION_norm', 'TOTAL_CRIME_norm']].fillna(value=0)
data[data.isnull().any(axis=1)]


,Pop_1E,MaleEN,FemEN,PopUnd18N,PopOv65N,HispN,NotHispN,WhiteN,BlackN,NativeN,...,PopOver25,EduOtherN,EduBachelor,EduGradProf,GardenCounts,HasGarden,FELONY_norm,MISDEMEANOR_norm,VIOLATION_norm,TOTAL_CRIME_norm
NTA,,,,,,,,,,,,,,,,,,,,,
park-cemetery-etc-Manhattan,1765.0,0.87762,0.12238,0.003399,0.094051,0.202833,0.797167,0.218697,0.511615,0.007932,...,1677.0,0.938581,0.042338,0.019082,2.0,1.0,0.760340,1.542210,0.287252,2.589802
Rikers Island,7503.0,0.93216,0.06784,0.035586,0.004398,0.298547,0.701453,0.067973,0.599227,0.001866,...,5322.0,0.985720,0.006576,0.007704,0.0,0.0,0.411036,0.278822,0.002532,0.692390


In [13]:
data[data.columns[data.isnull().any(axis=0)]][data.isnull().any(axis=1)]
# null values are for housing features
# will drop them

,HHINC_U10N,HHINC_10t14N,HHINC_15t24N,HHINC_25t34N,HHINC_35t49N,HHINC_50t74N,HHINC_75t99N,HHINC_100t149N,HHINC_150t199N,HHINC_200plN,CLF_UnemployedN
NTA,,,,,,,,,,,
park-cemetery-etc-Manhattan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.461774
Rikers Island,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
data.shape

(193, 39)

In [15]:
data.dropna(axis='index', inplace=True)
data.shape

(191, 39)

In [16]:
data_pop = data.copy()
data_pop.to_csv('../data/Norm_Data_Final_with_Pop.csv')

In [17]:
# drop population columns
data.columns

Index(['Pop_1E', 'MaleEN', 'FemEN', 'PopUnd18N', 'PopOv65N', 'HispN',
       'NotHispN', 'WhiteN', 'BlackN', 'NativeN', 'AsianN', 'PacificN',
       'OtherRace1N', 'OtherRace2N', 'HHPop', 'HHINC_U10N', 'HHINC_10t14N',
       'HHINC_15t24N', 'HHINC_25t34N', 'HHINC_35t49N', 'HHINC_50t74N',
       'HHINC_75t99N', 'HHINC_100t149N', 'HHINC_150t199N', 'HHINC_200plN',
       'Pop16yrsOver', 'Labor_forceN', 'CivilianLF', 'CLF_UnemployedN',
       'PopOver25', 'EduOtherN', 'EduBachelor', 'EduGradProf', 'GardenCounts',
       'HasGarden', 'FELONY_norm', 'MISDEMEANOR_norm', 'VIOLATION_norm',
       'TOTAL_CRIME_norm'],
      dtype='object')

In [18]:
# delete pop columns so that its easier for modelling
data.drop(columns=['Pop_1E', 'PopOver25'], inplace=True)

In [19]:
data.to_csv('../data/Norm_Data_Final.csv')